<a href="https://colab.research.google.com/github/rahiakela/nlp-research-and-practice/blob/main/text-similarity-works/17_icd_10_code_and_common_keyword_highlight_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

In [ ]:
!pip -q install spacy
!python -m spacy download en_core_web_sm

!pip install pillow

!sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev
!pip install -U pdftotext
!pip install PyPDF2
!pip install fitz
!pip install PyMuPDF

In [ ]:
!pip install fuzzywuzzy

In [ ]:
import numpy as np
import pandas as pd
import re
import os
import sys
import glob
import difflib
import pickle
import json
from pathlib import Path

import fitz
import pdftotext
from PyPDF2 import PdfFileReader, PdfReader, PdfFileWriter, PdfWriter

import spacy
from spacy.matcher import PhraseMatcher
from spacy.lang.en import English

import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
!mkdir -p input_files

In [ ]:
(128/255,128/255,0)

(0.5019607843137255, 0.5019607843137255, 0)

##Core Classes

In [ ]:
class Highlighter:
  def __init__(self):
    # loading and updating patterns for ICD-10 code
    self.nlp_code10 = English()
    self.nlp_code10.add_pipe("entity_ruler").from_disk("icd10_code_patterns-v5.jsonl")

    # loading stop words
    self.stop_words = set(stopwords.words('english'))

    # define icd-10 code dataset
    self.code_df = pd.read_csv("icd_10_code_and_keywords_v2.csv")

    # define required directory path
    self.PDF_FILES_PATH = "pdf-files"
    self.TXT_FILES_PATH = "txt-files"
    self.OUTPUT_FILES_PATH = "output"
    create_directory(self.PDF_FILES_PATH)
    create_directory(self.TXT_FILES_PATH)
    create_directory(self.OUTPUT_FILES_PATH)

  def split_pdf(self, pdf_path):
      pdf_in_file = open(pdf_path, "rb")
      pdf = PdfReader(pdf_in_file)
      pdf_list = []
      for page in range(len(pdf.pages)):
          input_pdf = PdfReader(pdf_in_file)
          output = PdfWriter()
          #output.addPage(input_pdf.getPage(page))
          output.add_page(input_pdf.pages[page])
          with open(f"{self.PDF_FILES_PATH}/page-{page}.pdf", "wb") as outputStream:
              output.write(outputStream)
              pdf_list.append(f"page-{page}.pdf")
      return pdf_list

  def extract_text_from_pdf(self, pdf_list):
    txt_file_list = []
    i = 0
    for pdf_file in pdf_list:
      with open(os.path.join(self.PDF_FILES_PATH, pdf_file), "rb") as f:
        pdf = pdftotext.PDF(f)

      # Read all the text into one string
      pdf_text = "\n\n".join(pdf)

      # write text into file
      with open(f"{self.TXT_FILES_PATH}/page-{str(i)}.txt", "a") as f:
        f.write(pdf_text)
      txt_file_list.append(f"{self.TXT_FILES_PATH}/page-{str(i)}.txt")
      i += 1
    self.text_list = txt_file_list
    return txt_file_list

  def highlight_icd_code(self, icd10_code_dict, match_threshold=30, coordinate=False, pdf_file_name=None, cords_file_name=None):
      pdf_file = fitz.open(pdf_file_name)
      # create file to write coordinate
      txt_output_file_name = open(f"{self.OUTPUT_FILES_PATH}/{cords_file_name}", "a")
      json_data = []
      # add file header 
      txt_output_file_name.write("| Page | Found Code | Actual ICD10-Code | ICD 10 description | Common Words | Matched paragraph | confidence |\n ")

      def highlight_code(page_obj, p_highlight, match_score):
        # highlight code if threshold is more than match threshold and common words exists
        if match_score >= match_threshold:
          page_highlight = page_obj.add_highlight_annot(p_highlight)
          page_highlight.set_colors(stroke=[0.66, 1, 0.07])  # light green
          page_highlight.update()
        else:
          # highlight and set color coding dont have common words
          page_highlight = page.add_highlight_annot(highlight)
          page_highlight.set_colors(stroke=[0.92, 0.59, 0.48]) # dark salmon
          page_highlight.update()

      def highlight_common_words(page_obj, common_words_coord_dict):
        highlight_coords_dict = {}
        highlighted_word_list = []
        for common_word, common_word_coord in common_words_coord_dict.items():
          highlight = page_obj.add_highlight_annot(common_word_coord)
          highlight.update()
          highlight_coords_dict[common_word] = common_word_coord
          highlighted_word_list.append(common_word)
        return highlight_coords_dict, highlighted_word_list
              
      for page_num, page in enumerate(pdf_file):
        # highlight ICD-10 code
        if page_num in icd10_code_dict:
          for code in icd10_code_dict[page_num]:
            highlight_list = page.search_for(code)
            print(f"Page-{page_num}, Code: {code}, Coordinate: {highlight_list}")
            num_page = page_num + 1
            for highlight in highlight_list:
              json_data_object = {}
              keyword = self.get_keyword(code)
              # get match score and common words coordinate
              all_match_keyword_list = self.get_best_token_match(code, page, highlight[3], match_threshold)

              # highlight and set color coding dont have common words
              if not all_match_keyword_list:
                page_highlight = page.add_highlight_annot(highlight)
                page_highlight.set_colors(stroke=[0.92, 0.59, 0.48]) # dark salmon
                page_highlight.update()
                # prepare json object
                json_data_object[f"page-{num_page}"] = {
                  "icd10_code": code,
                  "page_icd10_code": reverse_code_pattern(code),
                  "page_icd10_code_coord": f"{highlight}",
                  "icd10_code_desc": keyword if keyword else "UNVALIDATED",
                  "page_icd_common_words": "UNVALIDATED",
                  "page_icd_keywords_coord": {},
                  "page_icd_keywords_found": "N",
                  "match_confidence": 0
                } 
                json_data.append(json_data_object)
                # log data to file
                code_cors_output = (f"|Page-{num_page} | {code} | {reverse_code_pattern(code)}"
                f"| {keyword if keyword else 'No keyword found'} | 'UNVALIDATED' | 0 |")
                txt_output_file_name.write("%s\n" % code_cors_output)
                if coordinate:
                  txt_output_file_name.write("%s\n" % f"|{reverse_code_pattern(code)}:{highlight}|")

              for match_keyword_dict in all_match_keyword_list:
                # highlight ICD-10 code  
                highlight_code(page, highlight, match_keyword_dict["score"])
                # highlight common words
                highlight_coords_dict = {}
                highlight_coords_dict, highlighted_word_list = highlight_common_words(page, match_keyword_dict["common_words_coords"])
                # write all info into text file
                if match_keyword_dict["score"] >= match_threshold:
                  #| Page | Found Code | Actual ICD10-Code | ICD 10 description | Common Words | confidence | 
                  code_cors_output = (f"|Page-{num_page} | {code} | {reverse_code_pattern(code)}" 
                  f" | {keyword if keyword else 'No keyword found'}" 
                  f" | {match_keyword_dict['common_words'] if len(match_keyword_dict['common_words']) > 0 else 'UNVALIDATED'}"
                  f" | {match_keyword_dict['paragraph']} | {match_keyword_dict['score']} |")
                  txt_output_file_name.write("%s\n" % code_cors_output)
                else:
                  code_cors_output = (f"|Page-{num_page} | {code} | {reverse_code_pattern(code)} | {keyword if keyword else 'No keyword found'}"
                  f" | {match_keyword_dict['common_words'] if len(match_keyword_dict['common_words']) > 0 else 'UNVALIDATED'}"
                  f" | {match_keyword_dict['paragraph']} | {match_keyword_dict['score']} |")
                  txt_output_file_name.write("%s\n" % code_cors_output)
                if coordinate:
                  txt_output_file_name.write("%s\n" % f"|{reverse_code_pattern(code)}:{highlight}|")
                  txt_output_file_name.write("%s\n" % f"|{highlight_coords_dict}|")

                # prepare json object
                json_data_object[f"page-{num_page}"] = {
                  "icd10_code": code,
                  "page_icd10_code": reverse_code_pattern(code),
                  "page_icd10_code_coord": f"{highlight}",
                  "icd10_code_desc": keyword if keyword else "UNVALIDATED",
                  "page_icd_common_words": match_keyword_dict["common_words"] if len(match_keyword_dict["common_words"]) > 0 else 'UNVALIDATED',
                  "page_icd_keywords_coord": f"{highlight_coords_dict}",
                  "page_icd_keywords_found": "Y" if len(match_keyword_dict["common_words"]) > 0 else "N",
                  "match_confidence": match_keyword_dict['score']
                }
                json_data.append(json_data_object)
              txt_output_file_name.write("\n") # add extra line on every match code

      # build and write json file
      json_data_dump_file = f"{self.OUTPUT_FILES_PATH}/{pdf_file_name.split('/')[1].split('.')[0]}.json"
      with open(json_data_dump_file, "w") as json_out_file:
        json.dump(json_data, json_out_file)

      # close log file
      txt_output_file_name.close()
      # create highlighted pdf file
      pdf_output_file_name = f"{self.OUTPUT_FILES_PATH}/{pdf_file_name.split('/')[1].split('.')[0]}_output.pdf"
      pdf_file.save(pdf_output_file_name, garbage=4, deflate=True, clean=True)

      return pdf_output_file_name, cords_file_name

  def search_icd_code(self, txt_list):
    pdf_page_vocab = {}
    for txt_file in txt_list:
      with open(txt_file, "r") as f:
        page_txt = f.read()

        # check the page that have line number instead of code
        index_page = False
        if re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", page_txt):
          index_page = True

        doc = self.nlp_code10(page_txt)
        code_list = []
        for ent in doc.ents:
          if index_page:
            # check the code contain letter "L"
            if re.search("(L[0-9]+)", ent.text):
              continue
            else:
              code_list.append(ent.text)
          else:
            code_list.append(ent.text)

        #code_list = [ent.text for ent in doc.ents if not re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", ent.text)]
        if len(code_list) != 0:
          page_number = int(txt_file.split("/")[1].split(".")[0].split("-")[1])
          pdf_page_vocab[page_number] = list(set(code_list)) 
          # print(f"Page[{txt_file.split('/')[1]}]: {code_list}")
    return pdf_page_vocab
    
  def get_best_token_match(self, p_code, page_obj, code_y2_coord, match_threshold):
    all_match_keyword_list = []
    # Step 1: reverse code pattern
    reversed_icd_code = reverse_code_pattern(p_code)
    # Step 2: fetch keyword based on code 
    keyword = self.get_keyword(reversed_icd_code)
    print(f"keyword: {keyword}")
    # Step 3: get code paragraph  
    code_paragraph_list = self.get_paragraph(page_obj, code_y2_coord)
    #print(f"code_paragraph-2: {code_paragraph}")
    # Step 4: get common words 
    for code_paragraph in code_paragraph_list:
      match_keyword_dict = {}
      common_words = self.get_common_words(keyword, code_paragraph)
      if len(common_words) > 0:
        match_keyword_dict["common_words"] = common_words 
        # Step 4: get best token match ratio 
        clean_paragraph = " ".join(self.clean_text(code_paragraph))
        match_keyword_dict["paragraph"] = clean_paragraph 
        match_score = fuzz.token_set_ratio(keyword, clean_paragraph) if fuzz.token_set_ratio(keyword, clean_paragraph) > match_threshold else 0
        match_keyword_dict["score"] = match_score 
        # Step 5: build common words coordinate dict
        common_words_coord_dict = {}
        for common_word in common_words:
          highlight_list = page_obj.search_for(common_word)
          for highlight in highlight_list:
            # get common word y2 coord value
            common_word_y2_coords = highlight[3]
            if (code_y2_coord-20) <= common_word_y2_coords <= (code_y2_coord + 20):
              common_words_coord_dict[common_word] = highlight
        match_keyword_dict["common_words_coords"] = common_words_coord_dict
        all_match_keyword_list.append(match_keyword_dict)
    return all_match_keyword_list
  
  def get_keyword(self, p_code):
    keyword = ""
    # reverse code if required
    code = reverse_code_pattern(p_code)
    # get keyword from dataset
    keyword_list = list(self.code_df.loc[self.code_df["Code"] == code]["Keyword"])
    if len(keyword_list) > 0:
      keyword = keyword_list[0]
    return keyword

  def get_paragraph(self, page_obj, code_y2_coord):
    code_paragraph_list = []
    page_content = page_obj.get_text("blocks", sort=False)
    #y_coord_list = [content[3] for content in page_content]
    #closed_coord = find_nearest(y_coord_list, code_y2_coord)
    #print(f"closet_y_coords: {closed_coord}")
    for content in page_content:
      if content[1] <= code_y2_coord <= content[3]:
        if len(self.clean_text(content[4])) > 0:
          code_paragraph = content[4]
          code_paragraph_list.append(code_paragraph)
          print(f"paragraph_coord: {content[0], content[1], content[2], content[3]}")
          #print(f"code_paragraph: {code_paragraph}")
    return code_paragraph_list

  def get_paragraph4(page_obj, code_y2_coord):
    code_paragraph = None
    page_content = page_obj.get_text("blocks", sort=False, flags=11)
    for content in page_content:
      #print(f"1content[4]: {content[4]}")
      if content[1] <= code_y2_coord <= content[3]:
        # print(f"2content[4]: {content[4]}")
        if len(clean_text(content[4])) > 0:
          print(f"Para Coord: {content[0], content[1], content[2], content[3]}")
          code_paragraph = content[4]
          print(f"Paragraph: {code_paragraph}")
          #print("--" * 10)
    return code_paragraph
    
  def get_paragraph3(self, page_obj, code_y2_coord):
    code_paragraph = None
    page_content = page_obj.get_text("blocks", sort=False)
    y_coord_list = [content[3] for content in page_content]
    closed_coord = find_nearest(y_coord_list, code_y2_coord)
    print(f"closet_y_coords: {closed_coord}")
    for content in page_content:
      if (closed_coord-20) <= content[3] <= (closed_coord + 20):
        if len(clean_text(content[4])) > 0:
          code_paragraph = content[4]
          print(f"paragraph_coord: {content[0], content[1], content[2], content[3]}")
          print(f"code_paragraph: {code_paragraph}")
    return code_paragraph
    
  def get_paragraph2(self, page_obj, code_coords, y_coords_list):
    code_paragraph = None
    # find closet y coordinate value
    closet_y_coords = find_nearest(y_coords_list, value=code_coords)
    page_content = page_obj.get_text("blocks", sort=False)
    for content in page_content:
      print(f"closet_y_coords: {closet_y_coords}")
      if (closet_y_coords-10) <= content[1] <= (closet_y_coords + 10):
        print(f"content[1]: {content[1]}")
        print(f"content[4]: {content[4]}")
        if len(self.clean_text(content[4])) > 0:
          code_paragraph = content[4]
          print(f"code_paragraph-1: {code_paragraph}")
          print(f"paragraph_coord: {content[0], content[1], content[2], content[3]}")
    return code_paragraph

  def get_common_words(self, sent1, sent2):
    clean_token1 = self.clean_text(sent1)
    clean_token2 = self.clean_text(sent2)
    token_set1 = set(clean_token1)
    token_set2 = set(clean_token2)

    common_word_set = set()
    def get_common(token_set1, token_set2):
      for w1 in token_set1:
        for w2 in token_set2:
          if w1.lower() == w2.lower():
            common_word_set.add(w1)
    get_common(token_set1, token_set2)
    get_common(token_set2, token_set1)
    return list(common_word_set)

  def clean_text(self, sent):
    # tokenize sentence
    sent1 = word_tokenize(sent)
    # filter stop words
    filtered_sent = [w for w in sent1 if not w.lower() in stop_words]
    filtered_sent = [w for w in filtered_sent if re.sub(re.compile('\W'), '', w)]
    clean_tokens = []
    for token in filtered_sent:
      if token.find("-"):
        tokens = token.split("-")
        clean_tokens.extend(tokens)
      else:
        clean_tokens.append(token)
    return clean_tokens

  def get_best_token_match2(self, p_code, num_page, match_threshold):
    # Step 1: reverse code pattern
    reversed_icd_code = reverse_code_pattern(p_code)
    # Step 2: fetch keyword based on code 
    keyword = self.get_keyword(reversed_icd_code)
    # Step 3: prepare sentence list 
    sentence_list = get_sentence_list(self.text_list, num_page)
    # Step 4: get best match token ratio or wratio
    match_list = [
      (sentence, fuzz.token_set_ratio(keyword, sentence)) 
      for sentence in sentence_list if fuzz.token_set_ratio(keyword, sentence) > match_threshold
    ]
    match_found = sort_tuple(match_list)[0] if len(sort_tuple(match_list)) > 0 else ("NA", 0)
    return match_found, sentence_list,

def is_common_word_coords_nearby(page_obj, code_coord, keyword, match_sent):
  is_closest_coord = False
  # get common word list
  common_words = get_common_words(keyword, match_sent)
  if len(common_words) > 0:
    # get common word coordinate
    highlight_list = page_obj.search_for(common_words[0])
    #y_coords_list = [highlight[1] for highlight in highlight_list]
    # find closet y coordinate value
    #closet_y_coords = find_nearest(y_coords_array, value=code_coord)
    # check, common word coordinate within range of code coordinate
    # if (code_coord-10) <= closet_y_coords <= (code_coord + 10):
    if (code_coord-10) <= highlight_list[0][1] <= (code_coord + 10):
      is_closest_coord = True
  return is_closest_coord

def find_nearest(array, value):
  array = np.asarray(array)
  idx = (np.abs(array - value)).argmin()
  return array[idx]

def get_common_words(sent1, sent2):
  clean_token1 = clean_text(sent1)
  clean_token2 = clean_text(sent2)
  token_set1 = set(clean_token1)
  token_set2 = set(clean_token2)

  common_word_set = set()
  def get_common(token_set1, token_set2):
    for w1 in token_set1:
      for w2 in token_set2:
        if w1.lower() == w2.lower():
          common_word_set.add(w1)
  get_common(token_set1, token_set2)
  get_common(token_set2, token_set1)
  return list(common_word_set)

def clean_text(sent):
  # tokenize sentence
  sent1 = word_tokenize(sent)
  # filter stop words
  filtered_sent = [w for w in sent1 if not w.lower() in stop_words]
  filtered_sent = [w for w in filtered_sent if re.sub(re.compile('\W'), '', w)]
  clean_tokens = []
  for token in filtered_sent:
    if token.find("-"):
      tokens = token.split("-")
      clean_tokens.extend(tokens)
    else:
      clean_tokens.append(token)
  return clean_tokens

def get_sentence_line(p_code, sentence_list):
  line_list = [(line + 1) for line, sent in enumerate(sentence_list) if p_code in sent]
  return line_list

def get_sentence_list(text_list, num_page):
  with open(f"{text_list[num_page]}", "r") as f:
    lines = [line.rstrip('\n') for line in f]
    sentence_list = [" ".join(line.split()) for line in lines] # Remove extra spaces, tabs, and line breaks
  return sentence_list

def reverse_code_pattern(p_code):
  orig_code = p_code

  # check for code contains space(" ")
  tmp_code = orig_code.split(" ")
  if len(tmp_code) > 1:
    orig_code = f"{tmp_code[0].strip()}.{tmp_code[1].strip()}"

  # check for code contains dot(".")
  tmp_code = p_code.split(".")
  if len(tmp_code) > 1:
    orig_code = f"{tmp_code[0].strip()}.{tmp_code[1].strip()}"
  
  # check for code contains comma(",")
  tmp_code = p_code.split(",")
  if len(tmp_code) == 2:
    orig_code = f"{tmp_code[0].strip()}.{tmp_code[1].strip()}"
  elif len(tmp_code) == 2:
    orig_code = f"{tmp_code[0].strip()}.{tmp_code[2].strip()}"

  # handle if the first char of code is missing
  alphabats = {"Z": "2", "B": "8", "O": "0", "S": "5", "l": "1", "G": "6", "o": "9", "i": "1"}
  for key, val in alphabats.items():
    # replcae char on 0 index
    if orig_code.find(val) == 0:
      #orig_code = orig_code.replace(val, key)
      orig_code = replacer(orig_code, key, 0)
    # replcae char on 1 index
    if orig_code.find(key) == 1:
      orig_code = replacer(orig_code, val, 1)
      # replcae char on 2 index
      if orig_code.find(key) == 2:
        orig_code = replacer(orig_code, val, 2)
      break

  return orig_code

def replacer(s, newstring, index, nofail=False):
  # raise an error if index is outside of the string
  if not nofail and index not in range(len(s)):
      raise ValueError("index outside given string")

  # if not erroring, but the index is still not in the correct range..
  if index < 0:  # add it to the beginning
      return newstring + s
  if index > len(s):  # add it to the end
      return s + newstring

  # insert the new string between "slices" of the original
  return s[:index] + newstring + s[index + 1:]

def sort_tuple(p_tup):
  return (sorted(p_tup, key = lambda x: x[1], reverse=True)) 

def create_directory(dir_name):
  if not os.path.exists(dir_name):
    os.makedirs(dir_name)

##Keyword Matching & Highlighting 

- Step 1 - Z87.5
- Step 2 - Personal history of complications of pregnancy, childbirth and the puerperium
- Step 3 - Page keyword
- Step 4 - calculate cosine similirity
- Step 5 - "Green" > 60% otherwise "Yellow"

In [ ]:
!rm -rf input_files
!mkdir -p input_files
!rm -rf output
!mkdir -p output

In [ ]:
def purge(file_path):
  for f in glob.glob(file_path):
    os.remove(f)

In [ ]:
# Step-0: create highlighter instance
INPUT_PDF_FILES_PATH = "input_files"

highlighter = Highlighter()

In [ ]:
%%time

for pdf_file in os.listdir(INPUT_PDF_FILES_PATH):
  pdf_file_name = f"{INPUT_PDF_FILES_PATH}/{pdf_file}"
  cords_file_name = f"{pdf_file_name.split('/')[1].split('.')[0]}_cords.txt"

  # Step-1: splitting pdf file
  pdf_list = highlighter.split_pdf(pdf_file_name)

  # Step-2: Extracting text from pdf
  txt_list = highlighter.extract_text_from_pdf(pdf_list)

  # Step-3: Searching ICD-10 code
  icd10_code_dict = highlighter.search_icd_code(txt_list)

  # Step-4: Highlighting ICD-10 code into pdf
  pdf_output_file, txt_output_file = highlighter.highlight_icd_code(icd10_code_dict,
                                                                    match_threshold=35,
                                                                    coordinate=True,
                                                                    pdf_file_name=pdf_file_name,
                                                                    cords_file_name=cords_file_name)
  print(f"File[{pdf_output_file}] is saved after highlighting ICD-10 code")
  print(f"Highlighted coordinates are saved into [{txt_output_file}] file.")

  # remove all pdf and text files
  purge("pdf-files/*.pdf")
  purge("txt-files/*.txt")
  pdf_list = []
  txt_list = []

In [ ]:
purge("pdf-files/*.pdf")
purge("txt-files/*.txt")

In [ ]:
!zip output.zip output/*.*

In [ ]:
!zip pdf_text_output.zip pdf-files/*.* txt-files/*.*

In [ ]:
# Step-1: spliting pdf file
pdf_file_name = "7page.pdf"
pdf_list = highlighter.split_pdf(pdf_file_name)

# Step-2: Extracting text from pdf
txt_list = highlighter.extract_text_from_pdf(pdf_list)

In [ ]:
# Step-3: Searching ICD-10 code
page_code10_dict = highlighter.search_icd_code(txt_list)

In [ ]:
code_df = pd.read_csv("icd_10_code_and_keywords_v2.csv")

##Text clean up

In [ ]:
with open(f"{txt_list[7]}", "r") as f:
  my_text = f.read()

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(my_text)
sentences = [sentence.text for sentence in doc.sents]

In [ ]:
sentences

In [ ]:
def text_preprocess(text_file):
  sentence_list = []
  stopwords = ["is", "a"]
  doc = nlp(my_text)
  sentences = [sentence.text for sentence in doc.sents]
  for sent in sentences:
    clean_text = " ".join(sent.split())  # Remove extra spaces, tabs, and line breaks
    clean_text = re.sub(f"[{re.escape(punctuation)}]", "", clean_text) # Remove punctuation
    clean_text = re.sub(r"\b[0-9]+\b\s*", "", clean_text)     # Remove numbers
    clean_text = " ".join([w for w in clean_text.split() if not w.isdigit()]) # Remove digits= Side effect: removes extra spaces
    clean_text = " ".join([w for w in clean_text.split() if w.isalpha()]) # Remove non-alphabetic characters= Side effect: removes extra spaces
    clean_text = re.sub(r"[^A-Za-z0-9\s]+", "", clean_text) # Remove all special characters and punctuation
    # Remove stopwords from a list
    tokens = clean_text.split()
    clean_tokens = [t for t in tokens if not t in stopwords]
    clean_text = " ".join(clean_tokens)
    # Remove short tokens
    tokens = clean_text.split()
    clean_tokens = [t for t in tokens if len(t) > 3]
    clean_text = " ".join(clean_tokens)
    # Remove repeated characters
    clean_text = re.sub(r'(.)\1{3,}',r'\1', clean_text)
    if len(clean_text) > 0:
      sentence_list.append(clean_text)
  return sentence_list

In [ ]:
sentence_list = text_preprocess(my_text)
sentence_list

In [ ]:
with open(f"{txt_list[7]}", "r") as f:
  lines = [line.rstrip('\n') for line in f]
  sentence_list = [" ".join(line.split()) for line in lines] # Remove extra spaces, tabs, and line breaks
sentence_list

In [ ]:
def get_best_match(sentence_list, keyword):
  print("#"*10)
  print(f"Matching for : {keyword}")
  print()
  ratios = [fuzz.ratio(keyword, sentence) for sentence in sentence_list]
  best_match = sentence_list[ratios.index(max(ratios))]
  print(f"Best match: {fuzz.ratio(keyword, best_match)} | {best_match}")
  print(f"Before Match: {fuzz.ratio(keyword, sentence_list[sentence_list.index(best_match) - 1])} | {sentence_list[sentence_list.index(best_match) - 1]}")
  print(f"After Match: {fuzz.ratio(keyword, sentence_list[sentence_list.index(best_match) + 1])} | {sentence_list[sentence_list.index(best_match) + 1]}")
  print()

  p_ratios = [fuzz.partial_ratio(keyword, sentence) for sentence in sentence_list]
  p_best_match = sentence_list[p_ratios.index(max(p_ratios))]
  print(f"Partial Best match: {fuzz.partial_ratio(keyword, p_best_match)} | {p_best_match}")
  print(f"Partial Before Match: {fuzz.partial_ratio(keyword, sentence_list[sentence_list.index(p_best_match) - 1])} | {sentence_list[sentence_list.index(p_best_match) - 1]}")
  print(f"Partial After Match: {fuzz.partial_ratio(keyword, sentence_list[sentence_list.index(p_best_match) + 1])} | {sentence_list[sentence_list.index(p_best_match) + 1]}")
  print()

In [ ]:
keyword_list = [get_keyword(code) for code in clean_icd10_code]
keyword_list

In [ ]:
[get_best_match(sentence_list, keyword) for keyword in keyword_list]

In [ ]:
def get_best_match(sentence_list, keyword_list):
  for keyword in keyword_list:
    #for sentence in sentence_list:
    print("#"*10)
    print(f"Matching for : {keyword}")
    match_list = process.extract(keyword, sentence_list, scorer = fuzz.ratio, limit = 3)
    for match_found in match_list:
      print(f"{match_found[0]} | {match_found[1]}")
    print()

In [ ]:
get_best_match(sentence_list, keyword_list)
#process.extract(query, choices, scorer = fuzz.partial_ratio, limit = 2)

In [ ]:
sentence_list.index("Moderate Obstructive Sleep Apnea .")

25

In [ ]:
for idx, sent in enumerate(sentence_list):
  print(f"{idx}>{sent}")

##All Together

In [ ]:
def get_keyword(p_code):
  keyword = ""
   # reverse code if required
  code = reverse_code_pattern(p_code)
  # get keyword from dataset
  keyword_list = list(code_df.loc[code_df["Code"] == code]["Keyword"])
  if len(keyword_list) > 0:
    keyword = keyword_list[0]
  return keyword

In [ ]:
# Diagnosis: Hypertriglyceridemia, sporadic (E78.3)
get_keyword("E78.3")

'Hyperchylomicronemia'

In [ ]:
get_keyword("R03.0")
# Hypertriglyceridemia

'Elevated blood-pressure reading, w/o diagnosis of htn'

In [ ]:
clean_icd10_code = [reverse_code_pattern(code) for code in page_code10_dict[7]]
clean_icd10_code

['Z20.822', 'E78.3', 'R05.9']

Finding for `Hypersomnia, unspecified`:
44 > Hypoxemia
65 > (central sleep apnea (G47.31) CoHypersomnia, unspecified (G47.10)

In [ ]:
with open(f"{txt_list[15]}", "r") as f:
  lines = [line.rstrip('\n') for line in f]
  sentence_list = [" ".join(line.split()) for line in lines] # Remove extra spaces, tabs, and line breaks

In [ ]:
def sort_tuple(tup):
  return(sorted(tup, key = lambda x: x[1], reverse=True)) 

In [ ]:
[code for code in page_code10_dict[21]]

In [ ]:
[(code, get_keyword(code)) for code in page_code10_dict[15]]

[('Z00.00', 'Encntr for general adult medical exam w/o abnormal findings')]

In [ ]:
def get_best_token_match(p_code, num_page):
  # Step 1: reverse code pattern
  reversed_icd_code = reverse_code_pattern(p_code)
  # Step 2: fetch keyword based on code 
  keyword = get_keyword(reversed_icd_code)
  # Step 3: prepare sentence list 
  with open(f"{txt_list[num_page]}", "r") as f:
    lines = [line.rstrip('\n') for line in f]
    sentence_list = [" ".join(line.split()) for line in lines] # Remove extra spaces, tabs, and line breaks
  # Step 4: get best token match ratio 
  match_list = [(sentence, fuzz.token_set_ratio(keyword, sentence)) for sentence in sentence_list if fuzz.token_set_ratio(keyword, sentence) > 40]
  return sort_tuple(match_list)

In [ ]:
match_list = [get_best_token_match(code, 15) for code in page_code10_dict[15]]

In [ ]:
match_list

[[('Diagnosis: Routine general medical examination at a health care facility (Z00.00)',
   42)]]

In [ ]:
match_list = get_best_token_match("E78.3", 7)
match_list

[]

In [ ]:
match_list[0]

'Diagnosis Fever (R50.9)'

In [ ]:
match_list[1]

47

In [ ]:
def clean_text(sent):
  # tokenize sentence
  sent1 = word_tokenize(sent)
  # filter stop words
  filtered_sent = [w for w in sent1 if not w.lower() in stop_words]
  filtered_sent = [w for w in filtered_sent if re.sub(re.compile('\W'), '', w)]
  return filtered_sent

In [ ]:
def clean_text(sent):
  # tokenize sentence
  sent1 = word_tokenize(sent)
  # filter stop words
  filtered_sent = [w for w in sent1 if not w.lower() in stop_words]
  filtered_sent = [w for w in filtered_sent if re.sub(re.compile('\W'), '', w)]
  clean_tokens = []
  for token in filtered_sent:
    if token.find("-"):
      tokens = token.split("-")
      clean_tokens.extend(tokens)
    else:
      clean_tokens.append(token)
  return clean_tokens

In [ ]:
# Diagnosis Sprain of calcaneofibular (ligament) of right ankle, initial encounter (S93.411A)
clean_token = clean_text("Overweight (BMI 25.0 - 29.9) (E66.3)")
clean_token

['Overweight', 'BMI', '25.0', '29.9', 'E66.3']

In [ ]:
def get_common_words(sent1, sent2):
  clean_token1 = clean_text(sent1)
  clean_token2 = clean_text(sent2)
  token_set1 = set(clean_token1)
  token_set2 = set(clean_token2)

  common_word_set = set()
  def get_common(token_set1, token_set2):
    for w1 in token_set1:
      for w2 in token_set2:
        if w1.lower() == w2.lower():
          common_word_set.add(w1)
  get_common(token_set1, token_set2)
  get_common(token_set2, token_set1)
  
  return list(common_word_set)

In [ ]:
get_common_words(
  "Encntr for general adult medical exam w/o abnormal findings",
  "5/8/2020: Office Visit - Routine general medical examination at a health care facility (Z00.00), Hypertriglyceridemia, sporadic"
)

['general', 'medical']

In [ ]:
def get_common_words(sent1, sent2):
  clean_token1 = clean_text(sent1)
  clean_token2 = clean_text(sent2)
  token_set1 = set(clean_token1)
  token_set2 = set(clean_token2)
  for token in token_set1
  return list(token_set1 & token_set2)

In [ ]:
clean_tokens = []
for token in clean_token:
  if token.find("-"):
    tokens = token.split("-")
    clean_tokens.extend(tokens)
  else:
    clean_tokens.append(token)
clean_tokens

['Elevated', 'blood', 'pressure', 'reading', 'w/o', 'diagnosis', 'htn']

In [ ]:
def find_nearest(array, value):
  array = np.asarray(array)
  idx = (np.abs(array - value)).argmin()
  return array[idx]

In [ ]:
array = np.random.random(10)
print(array)

[0.93361323 0.10674056 0.81360362 0.26873547 0.7892294  0.36625687
 0.48473053 0.43495106 0.68754387 0.28034082]


In [ ]:
print(find_nearest(array, value=0.35))

0.3662568679192558


##JSON

In [ ]:
json_data = {}

json_data[f"page-{1}"] = {
    "icd10_code": "R05.9",
    "page_icd10_code": "R05.9",
    "page_icd10_code_coord": "Rect(135.0170440673828, 307.5999450683594, 152.56533813476562, 315.6038513183594)",
    "icd10_code_desc": "Cough, unspecified",
    "page_icd_keywords_found": ['Cough'],
    "page_icd_keywords_coord": {'Cough': "Rect(104.86994934082031, 307.5999450683594, 124.26808166503906, 315.6038513183594)"},
    "keyword_match_found": "Y",
    "match_confidence": 45
}

json.dumps(json_data)

'{"page-1": {"icd10_code": "R05.9", "page_icd10_code": "R05.9", "page_icd10_code_coord": "Rect(135.0170440673828, 307.5999450683594, 152.56533813476562, 315.6038513183594)", "icd10_code_desc": "Cough, unspecified", "page_icd_keywords_found": ["Cough"], "page_icd_keywords_coord": {"Cough": "Rect(104.86994934082031, 307.5999450683594, 124.26808166503906, 315.6038513183594)"}, "keyword_match_found": "Y", "match_confidence": 45}}'

##Fitz

In [ ]:
def find_nearest(array, value):
  array = np.asarray(array)
  idx = (np.abs(array - value)).argmin()
  return array[idx]

In [ ]:
pdf_file = fitz.open("page-11.pdf")  #Create pdf file object
pdf_page_count = pdf_file.page_count   #var to hold page count
for page in range(pdf_page_count):  #notice that page starts with index 0
  page_obj = pdf_file[page] #Create page object
  #content_of_page = pdf_file.get_page_text(page) #Get page content
  match_word = "Z00.00" 
  content_of_page = page_obj.get_text("blocks", sort=False)  #get rect for all words

  coords_array = []
  y_coords_array = []
  for content in content_of_page:
    print(content)
    if content[4].replace("(", "").replace(")", "") == match_word:
      rect_comp = fitz.Rect(content[0],content[1],content[2],content[3])
      print(f"Line #{content[6]}-{content[4]} > {rect_comp}")
      coords_array.append(rect_comp)
      y_coords_array.append(content[1])
      
      #coord = fitz.Rect(content[1]).rect
      print(rect_comp[1])

      highlight = page_obj.add_highlight_annot(rect_comp)
      highlight.set_colors(stroke=[0.2, 1, 0.8])
      highlight.update()

  # find closet y coordinate value
  closet_y_coords = find_nearest(y_coords_array, value=74.320068359375)
  match_word = "Comprehensive" 
  for content in content_of_page:
    # print(word)
    if content[4].replace("(", "").replace(")", "") == match_word and (closet_y_coords-20) <= content[1] <= (closet_y_coords + 20):
      rect_comp = fitz.Rect(content[0],content[1],content[2],content[3])
      print(f"Line #{content[6]}-{content[4]} > {rect_comp}")
      highlight = page_obj.add_highlight_annot(rect_comp)
      highlight.set_colors(stroke=[1, 1, 0])
      highlight.update()

pdf_output_file_name = f"page_11_output.pdf"
pdf_file.save(pdf_output_file_name, garbage=4, deflate=True, clean=True)

In [ ]:
list1 = [74.320068359375, 237.0400390625, 409.8399658203125, 572.5599975585938]
curr_value = 74.320068359375 + 20
find_nearest(list1, value=curr_value)

74.320068359375

In [ ]:
curr_value = 74.320068359375 - 20
find_nearest(list1, value=curr_value)

74.320068359375

In [ ]:
def get_word_coordinate(page_obj, common_words):
  word_coords = {}
  content_of_page = page_obj.get_text("words", sort=False)  #get rect for all words
  for content in content_of_page:
    # print(word)
    for idx, common_word in enumerate(common_words):
      word_coord = {}
      curr_word = content[4].replace("(", "").replace(")", "")
      if curr_word == match_word:
        rect_comp = fitz.Rect(content[0],content[1],content[2],content[3])
        word_coord[common_word] = rect_comp
        word_coord["Line#"] = content[6]
        word_coords[idx] = word_coord
  return word_coords

In [ ]:
pdf_file = fitz.open("pdf-files/page-11.pdf")  #Create pdf file object
pdf_page_count = pdf_file.page_count   #var to hold page count
for page in range(pdf_page_count):  #notice that page starts with index 0
  page_obj = pdf_file[page] #Create page object
  print(get_word_coordinate(page_obj, ["general", "adult", "medical", "exam"]))

{0: {'general': Rect(242.856689453125, 65.00006103515625, 274.8216857910156, 72.00347900390625), 'Line#': 2}, 1: {'adult': Rect(242.856689453125, 65.00006103515625, 274.8216857910156, 72.00347900390625), 'Line#': 2}, 2: {'medical': Rect(242.856689453125, 65.00006103515625, 274.8216857910156, 72.00347900390625), 'Line#': 2}, 3: {'exam': Rect(242.856689453125, 65.00006103515625, 274.8216857910156, 72.00347900390625), 'Line#': 2}}


In [ ]:
def highlight_common_words(page_obj, y_coords_array, curr_y_coord, common_word_list):
  page_content = page_obj.get_text("words", sort=False)  #get rect for all words

  # find closet y coordinate value
  closet_y_coords = find_nearest(y_coords_array, value=curr_y_coord)
  match_word = "Comprehensive" 
  for content in page_content:
    for common_word in common_word_list:
      if content[4].replace("(", "").replace(")", "") == common_word and (closet_y_coords-20) <= content[1] <= (closet_y_coords + 20):
        rect_comp = fitz.Rect(content[0],content[1],content[2],content[3])
        print(f"Line #{content[6]}-{content[4]} > {rect_comp}")
        highlight = page_obj.add_highlight_annot(rect_comp)
        highlight.set_colors(stroke=[1, 1, 0])
        highlight.update()

In [ ]:
def get_keyword(p_code):
  keyword = ""
   # reverse code if required
  code = reverse_code_pattern(p_code)
  # get keyword from dataset
  keyword_list = list(code_df.loc[code_df["Code"] == code]["Keyword"])
  if len(keyword_list) > 0:
    keyword = keyword_list[0]
  return keyword

In [ ]:
def clean_text(sent):
  # tokenize sentence
  sent1 = word_tokenize(sent)
  # filter stop words
  filtered_sent = [w for w in sent1 if not w.lower() in stop_words]
  filtered_sent = [w for w in filtered_sent if re.sub(re.compile('\W'), '', w)]
  clean_tokens = []
  for token in filtered_sent:
    if token.find("-"):
      tokens = token.split("-")
      clean_tokens.extend(tokens)
    else:
      clean_tokens.append(token)
  return clean_tokens

In [ ]:
def get_common_words(sent1, sent2):
  clean_token1 = clean_text(sent1)
  clean_token2 = clean_text(sent2)
  token_set1 = set(clean_token1)
  token_set2 = set(clean_token2)

  common_word_set = set()
  def get_common(token_set1, token_set2):
    for w1 in token_set1:
      for w2 in token_set2:
        if w1.lower() == w2.lower():
          common_word_set.add(w1)
  get_common(token_set1, token_set2)
  get_common(token_set2, token_set1)
  
  return list(common_word_set)

In [ ]:
def get_paragraph(page_obj, code_coords):
  code_paragraph = None
  page_content = page_obj.get_text("blocks", sort=False)
  y_coord_list = [content[1] for content in page_content]
  closed_coord = find_nearest(y_coord_list, code_coords)
  print(f"closed_coord: {closed_coord}")
  for content in page_content:
    print(f"1-content[1]:{content[1]}")
    if (code_coords-50) <= content[1] <= (code_coords + 50):
      print(f"2-content[1]:{content[1]}")
      if len(clean_text(content[4])) > 0:
        code_paragraph = content[4]
        print(f"code_paragraph: {code_paragraph}")
  return code_paragraph

In [ ]:
flags = fitz.TEXT_PRESERVE_LIGATURES | fitz.TEXT_PRESERVE_WHITESPACE  # omits images
def get_paragraph(page_obj, code_coords, p_code):
  code_paragraph = None
  page_content = page_obj.get_text("blocks", sort=False)
  y_coord_list = [content[3] for content in page_content]
  print(f"Page coord: {page_obj.search_for(p_code)}")
  closed_coord = find_nearest(y_coord_list, code_coords)
  print(f"closed_coord: {closed_coord}")
  for content in page_content:
    print(f"1-content[1]:{content[1]}")
    if (closed_coord-20) <= content[3] <= (closed_coord + 20):
      print(f"2-content[1]:{content[1]}")
      if len(clean_text(content[4])) > 0:
        code_paragraph = content[4]
        print(f"paragraph coord: {content[0], content[1], content[2], content[3]}")
        print(f"code_paragraph: {code_paragraph}")
  return code_paragraph

In [ ]:
def get_paragraph(page_obj, code_coords):
  code_paragraph = None
  page_content = page_obj.get_text("blocks", sort=False)
  print(f"Code coord: {page_obj.search_for('R05.9')}")
  for content in page_content:
    print(f"Para coord:{content[0], content[1], content[2], content[3]}")
    print(f"content[4]:{content[4]}")
    print(clean_text(content[4]))
    print("#" * 10)
  return code_paragraph

In [ ]:
def get_paragraph(page_obj, code_y2_coord):
  code_paragraph = None
  page_content = page_obj.get_text("blocks", sort=False)
  y_coord_list = [content[3] for content in page_content]
  closed_coord = find_nearest(y_coord_list, code_y2_coord)
  for content in page_content:
    if (closed_coord-20) <= content[3] <= (closed_coord + 20):
      if len(clean_text(content[4])) > 0:
        code_paragraph = content[4]
  return code_paragraph

In [ ]:
def get_paragraph(page_obj, code_y2_coord):
  code_paragraph = None
  page_content = page_obj.get_text("blocks", sort=False)
  y_coord_list = [content[3] for content in page_content]
  # closed_coord = find_nearest(y_coord_list, code_y2_coord)
  for content in page_content:
    for y_coord in y_coord_list:
      if (y_coord-20) <= content[3] <= (y_coord + 20):
        if len(clean_text(content[4])) > 0:
          code_paragraph = content[4]
          print(code_paragraph)
          print("--" * 10)
  return code_paragraph

In [ ]:
def find_nearest(array, value):
  array = np.asarray(array)
  idx = (np.abs(array - value)).argmin()
  return array[idx]

In [ ]:
my_list = [31.44390869140625, 63.6053466796875, 80.64532470703125, 105.61761474609375,
           63.6043701171875, 114.96392822265625, 143.52392578125, 182.6453857421875,
           217.92388916015625, 238.08392333984375, 276.243896484375, 314.4043884277344,
           338.6448974609375, 352.0843811035156, 767.0497436523438]

find_nearest(my_list, 172.80389404296875)

182.6453857421875

In [ ]:
find_nearest(my_list, 284.6439208984375)

276.243896484375

In [ ]:
p_code = "R05.9"
code_coords = 172.80389404296875
pdf_file = fitz.open("22page.pdf")  #Create pdf file object
pdf_page_count = pdf_file.page_count   #var to hold page count
for page in range(pdf_page_count):  #notice that page starts with index 0
  page_obj = pdf_file[page] #Create page object
  paragraph = get_paragraph(page_obj, code_coords)
  print(paragraph)
  print("--" * 10)
  print(clean_text(paragraph))
  #common_words = get_common_words("Contact with and (suspected) exposure to COVID-19", paragraph)
  #print(common_words)

Assessment & Plan —_= Muneer; 12/28/2021 11:04 AM) 
Contact with and (suspected) 
exposure to COVID-19 (Z20.822) 
Current Plans 

--------------------
['Assessment', 'Plan', '—_=', 'Muneer', '12/28/2021', '11:04', 'Contact', 'suspected', 'exposure', 'COVID', '19', 'Z20.822', 'Current', 'Plans']


In [ ]:
(153.6400146484375 <= 172.80389404296875 <= 182.6453857421875)

True

In [ ]:
(276.6400146484375 <= 284.6439208984375 <= 314.4043884277344)

True

In [ ]:
def get_paragraph(page_obj, code_y2_coord):
  code_paragraph = None
  page_content = page_obj.get_text("blocks", sort=False, flags=11)
  for content in page_content:
    #print(f"1content[4]: {content[4]}")
    if content[1] <= code_y2_coord <= content[3]:
      # print(f"2content[4]: {content[4]}")
      if len(clean_text(content[4])) > 0:
        print(f"Para Coord: {content[0], content[1], content[2], content[3]}")
        code_paragraph = content[4]
        print(f"Paragraph: {code_paragraph}")
        #print("--" * 10)
  return code_paragraph

In [ ]:
# code_coords = 172.80389404296875
# code_coords =  284.6439208984375
code_coords = 359.52392578125
pdf_file = fitz.open("40page.pdf")  #Create pdf file object
pdf_page_count = pdf_file.page_count   #var to hold page count
for page in range(pdf_page_count):  #notice that page starts with index 0
  page_obj = pdf_file[page] #Create page object
  paragraph = get_paragraph(page_obj, code_coords)
  #print(paragraph)
  #print("--" * 10)
  #print(clean_text(paragraph))
  #common_words = get_common_words("Contact with and (suspected) exposure to COVID-19", paragraph)
  #print(common_words)

Para Coord: (28.5572566986084, 342.8800048828125, 440.1177062988281, 368.1639404296875)
Paragraph: Assessment & Plan (Tristan Guevara DO; 4/2/2020 12:52 PM) 
Fever (R50.9) 
Impression: 
x 6 days fever 102 better with tylenol - wife and daighter sick and resolved - fatigue /neadache 



In [ ]:
def get_best_token_match(p_code, page_obj, code_coords, match_threshold):
  # Step 1: reverse code pattern
  reversed_icd_code = reverse_code_pattern(p_code)
  # Step 2: fetch keyword based on code 
  keyword = get_keyword(reversed_icd_code)
  # Step 3: get code paragraph  
  code_paragraph = get_paragraph(page_obj, code_coords)
  # Step 4: get common words 
  common_words = get_common_words(keyword, paragraph)
  # Step 4: get best token match ratio 
  clean_paragraph = " ".join(clean_text(code_paragraph))
  match_score = fuzz.token_set_ratio(keyword, clean_paragraph) if fuzz.token_set_ratio(keyword, clean_paragraph) > match_threshold else 0
  # Step 5: build common words coordinate dict
  common_words_coord_dict = {}
  for common_word in common_words:
    highlight_list = page_obj.search_for(common_word)
    for highlight in highlight_list:
      # get y0 coord value
      y_coords = highlight[1]
      if (code_coords-20) <= y_coords <= (code_coords + 20):
        common_words_coord_dict[common_word] = highlight
  print(common_words_coord_dict)
  print("#" * 10)
  #print(f"Page-{num_page}, Code: ({p_code})-{code_coords}, closet_y_coords:{closet_y_coords} Keyword: {keyword}, Match: {match_f}")
  return match_score, common_words, common_words_coord_dict

In [ ]:
pdf_file = fitz.open("36page.pdf")  #Create pdf file object
pdf_page_count = pdf_file.page_count   #var to hold page count
for page in range(pdf_page_count):  #notice that page starts with index 0
  page_obj = pdf_file[page] #Create page object
  match_score, common_words, common_words_coord_dict = get_best_token_match("Z00.00", page_obj, 54.60003662109375, 40)
  print(match_score, common_words, common_words_coord_dict)

{'general': Rect(169.1837615966797, 54.60003662109375, 197.1109161376953, 63.60443115234375), 'medical': Rect(204.7003631591797, 54.60003662109375, 233.25741577148438, 63.60443115234375)}
##########
41 ['general', 'medical'] {'general': Rect(169.1837615966797, 54.60003662109375, 197.1109161376953, 63.60443115234375), 'medical': Rect(204.7003631591797, 54.60003662109375, 233.25741577148438, 63.60443115234375)}


In [ ]:
pdf_file = fitz.open("38page.pdf")  #Create pdf file object
pdf_page_count = pdf_file.page_count   #var to hold page count
for page in range(pdf_page_count):  #notice that page starts with index 0
  page_obj = pdf_file[page] #Create page object
  match_score, common_words, common_words_coord_dict = get_best_token_match("Z00.00", page_obj, 360.1600036621094, 40)
  print(match_score, common_words, common_words_coord_dict)

{'general': Rect(106.54977416992188, 360.1600036621094, 130.4874725341797, 368.1639099121094), 'medical': Rect(137.50680541992188, 360.1600036621094, 161.8644256591797, 368.1639099121094)}
##########
41 ['general', 'medical'] {'general': Rect(106.54977416992188, 360.1600036621094, 130.4874725341797, 368.1639099121094), 'medical': Rect(137.50680541992188, 360.1600036621094, 161.8644256591797, 368.1639099121094)}


In [ ]:
pdf_file = fitz.open("38page.pdf")  #Create pdf file object
pdf_page_count = pdf_file.page_count   #var to hold page count
for page in range(pdf_page_count):  #notice that page starts with index 0
  page_obj = pdf_file[page] #Create page object
  match_score, common_words, common_words_coord_dict = get_best_token_match("Z00.00", page_obj, 120.63995361328125, 40)
  print(match_score, common_words, common_words_coord_dict)

{'general': Rect(60.47419738769531, 120.63995361328125, 84.62186431884766, 128.64385986328125), 'medical': Rect(91.91117858886719, 120.63995361328125, 116.8987808227539, 128.64385986328125)}
##########
41 ['general', 'medical'] {'general': Rect(60.47419738769531, 120.63995361328125, 84.62186431884766, 128.64385986328125), 'medical': Rect(91.91117858886719, 120.63995361328125, 116.8987808227539, 128.64385986328125)}


In [ ]:
def get_best_token_match(p_code, num_page, page_obj, code_coords, match_threshold):
    # Step 1: reverse code pattern
    reversed_icd_code = reverse_code_pattern(p_code)
    # Step 2: fetch keyword based on code 
    keyword = get_keyword(reversed_icd_code)
    # Step 3: prepare sentence list 
    sentence_list = get_sentence_list(txt_list, num_page)
    # Step 4: get best token match ratio 
    match_list = [
      (sentence, fuzz.token_set_ratio(keyword, sentence)) 
      for sentence in sentence_list if fuzz.token_set_ratio(keyword, sentence) > match_threshold
    ]

    match_found_list = []
    num_page = num_page + 1
    for match_f in match_list:
      highlight_list = page_obj.search_for(match_f[0])
      # prepare list for y0 coord value
      y_coords_list = [highlight[1] for highlight in highlight_list]
      #print(y_coords_list)
      # find closet y coordinate value
      if len(y_coords_list) > 0:
        closet_y_coords = find_nearest(y_coords_list, value=code_coords)
        # print(f"Code: {code_coords}, Y: {y_coords}")
        if (closet_y_coords-20) <= code_coords <= (closet_y_coords + 20):
          paragraph = get_paragraph(page_obj, code_coords)
          print(paragraph)
          print("#" * 10)
          match_found_list.append(match_f)
          #print(f"Page-{num_page}, Code: ({p_code})-{code_coords}, closet_y_coords:{closet_y_coords} Keyword: {keyword}, Match: {match_f}")
    return match_found_list, sentence_list

In [ ]:
pdf_file = fitz.open("22page.pdf")  #Create pdf file object
pdf_page_count = pdf_file.page_count   #var to hold page count
for page in range(pdf_page_count):  #notice that page starts with index 0
  page_obj = pdf_file[page] #Create page object
  match_found_list, sentence_list = get_best_token_match("Z00.00", 0, page_obj, 54.60003662109375, 40)
  #print(match_found_list)